In [1]:
import os
import sys
import numpy as np
import tensorrt as trt
import cv2 as cv
import torchvision
import torch
import time
from loguru import logger
from yolox_detector import YOLOX_DETECTOR
from classification import CLASSIFICATION
torch.cuda.init()


#detector box값 도출 해주는것 
def visual(frame,output, ratio, cls_conf=0.35):

    #아무런 output이 나오지 않을경우 
    if output is None:
        return None, None, None
    output = output.cpu()
    bboxes = output[:, 0:4]
    # preprocessing: resize
    bboxes /= ratio
    cls = output[:, 6] 
    scores = output[:, 4] * output[:, 5] 
    original_bboxes = []
    original_cls = []
    original_scores = []
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(cls[i])
        score = scores[i]
        
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])
        
        #사람 
        if cls_id == 0 and cls_conf < score :
            original_cls.append(cls_id)
            original_scores.append(score)
            original_bboxes.append([x_min,y_min,x_max,y_max])

    #output은 나왔지만 차량에 해당 하는 output이 없을경우 (사람만 포착이 되었을경우)     
    if len(original_bboxes) == 0 :
        return None, None, None

    return original_bboxes, original_cls, original_scores

#디텍션에서 나온 바운딩 박스값 잘라서 분류기 넣어 주기 위한것 
def crop_detect(frame,bboxes,scores):
    cut_img = []
    for i in range(len(bboxes)):
        box = bboxes[i]
        score = scores[i]
        if score < 0.5:
            continue
        x0 = abs(int(box[0]))
        y0 = abs(int(box[1]))
        x1 = abs(int(box[2]))
        y1 = abs(int(box[3]))
        img = frame[y0:y1,x0:x1]
        
        cut_img.append(img)
    return cut_img

#바운딩 박스 그려주기 (영상 저장 하고 싶을때 )
def vis(frame, boxes,cls_output):
    img = frame
    for i in range(len(boxes)):
        box = boxes[i]
        pred = int(cls_output[i][0])
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])

        if pred == 0 :
            category = categoty_name[0]
        if pred == 1 :
            category = categoty_name[1]
        if pred == 2 :
            category = categoty_name[2]
        color = [(0,0,255),(0,255,0),(200,200,255),(100,100,255)]
        text = '{}'.format(f'{category}')
        txt_color = (255, 255, 255)
        font = cv.FONT_HERSHEY_SIMPLEX

        txt_size = cv.getTextSize(text, font, 0.2, 1)[0]
        cv.rectangle(img, (x0, y0), (x1, y1), color[pred], 2)

        txt_bk_color = (0, 0, 0)
        cv.rectangle(
            img,
            (x0, y0 + 1),
            (x0 + txt_size[0] + 1, y0 + int(1.5*txt_size[1])),
            txt_bk_color,
            -1
        )
        cv.putText(img, text, (x0, y0 + txt_size[1]), font, 0.4, txt_color, thickness=1)


    return img

class Model:
    def __init__(self):
        self.trt_engine_path = '/DATA_17/engines/fp16/yoloxm_fp16_024.trt'
        self.trt_engine_path2 = '/DATA_17/ij/trt_inference/make_trt/helmet2.trt'
        self.yd = YOLOX_DETECTOR()
        self.cls = CLASSIFICATION()
    def initialize(self):
        self.yd.load(self.trt_engine_path)
        self.cls.load(self.trt_engine_path2)
         
    
    def proc(self,intput_path,output_path):
        cap = cv.VideoCapture(intput_path)
        width = cap.get(cv.CAP_PROP_FRAME_WIDTH)  # float
        height = cap.get(cv.CAP_PROP_FRAME_HEIGHT)  # float
        fps = cap.get(cv.CAP_PROP_FPS)

        save_path = output_path
        logger.info(f"video save_path is {save_path}")
        vid_writer = cv.VideoWriter(
            save_path, cv.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height))
        )

        print('fps',fps)

        cnt = 0

        if cap.isOpened() :    
            while True:
                cnt+=1
                #원본이미지 = frame 
                ret, frame = cap.read()
                if ret == False :
                    break
                if cnt < 0:
                    continue
    #             프레임수 특정값 이상일떄는 break 할때 
#                 if cnt > 10:
#                     break
                #현재 프레임
                num_frame = round(cap.get(cv.CAP_PROP_POS_FRAMES)) - 1
                print('frame : ',num_frame)


                #img stack
                input_data = self.yd.preprocess_multi([frame])

                #inference
                output_data = self.yd.inference(input_data)

                #postprocess
                outputs = self.yd.postprocess(output_data, class_agnostic=True)

                ratio = min(640 / frame.shape[0], 640 / frame.shape[1])

                original_bboxes, original_cls, original_scores = visual(frame,outputs[0], ratio, cls_conf=0.35)

                
                #박스 그리기 
                if not original_bboxes == None :
                    cut_img_list = crop_detect(frame,original_bboxes,original_scores)

                    if cut_img_list == None:
                        result_frame = frame 
                    else :    
                                          #img stack
                        cls_input_data = self.cls.preprocess_multi(cut_img_list)

                        #inference
                        cls_output = self.cls.inference(cls_input_data)
                        cls_output = self.cls.postprocess(cls_output)

                        result_frame = vis(frame, original_bboxes,cls_output)
                vid_writer.write(result_frame)
                ch = cv.waitKey(1)
                if ch == 27 or ch == ord("q") or ch == ord("Q"):
                    break



intput_path = './input/hat_ij.mp4'
output_path = './output/hat_ij.mp4'
categoty_name = {0 : "helmet", 1 : "head", 2 : "error"} 
m = Model()
m.initialize()
print('load_완료')
m.proc(intput_path,output_path)

print('#####################done########################')        

[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.

[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
2022-06-09 15:45:11.273 | INFO     | __main__:proc:126 - video save_path is ./output/hat_ij.mp4


load_완료
fps 29.99921036958811
frame :  0
pred 0
pred 1
frame :  1
pred 0
pred 1
frame :  2
pred 0
pred 1
frame :  3
pred 0
pred 1
frame :  4
pred 0
pred 1
frame :  5
pred 0
pred 1
frame :  6
pred 0
pred 1
frame :  7
pred 0
pred 0
frame :  8
pred 0
pred 0
frame :  9
pred 0
pred 0
frame :  10
pred 0
pred 0
frame :  11
pred 0
pred 0
frame :  12
pred 0
pred 1
frame :  13
pred 1
pred 0
frame :  14
pred 0
pred 0
frame :  15
pred 1
pred 0
frame :  16
pred 1
pred 0
frame :  17
pred 1
pred 0
frame :  18
pred 1
pred 0
frame :  19
pred 1
pred 0
frame :  20
pred 1
pred 0
frame :  21
pred 1
pred 0
frame :  22
pred 1
pred 0
frame :  23
pred 1
pred 0
frame :  24
pred 1
pred 0
frame :  25
pred 1
pred 0
frame :  26
pred 1
pred 0
frame :  27
pred 1
pred 0
frame :  28
pred 1
pred 0
frame :  29
pred 1
pred 0
frame :  30
pred 1
pred 0
frame :  31
pred 1
pred 0
frame :  32
pred 1
pred 0
frame :  33
pred 1
pred 0
frame :  34
pred 1
pred 0
frame :  35
pred 1
pred 0
frame :  36
pred 1
pred 0
frame :  37
pred 1

frame :  315
pred 0
pred 1
frame :  316
pred 0
pred 1
frame :  317
pred 0
pred 1
frame :  318
pred 0
pred 1
frame :  319
pred 0
pred 1
frame :  320
pred 0
pred 1
frame :  321
pred 0
pred 1
frame :  322
pred 0
pred 1
frame :  323
pred 0
pred 1
frame :  324
pred 0
pred 1
frame :  325
pred 1
pred 0
frame :  326
pred 1
pred 0
frame :  327
pred 1
pred 0
frame :  328
pred 1
pred 0
frame :  329
pred 1
pred 0
frame :  330
pred 1
pred 0
frame :  331
pred 0
pred 1
frame :  332
pred 0
pred 1
frame :  333
pred 0
pred 1
frame :  334
pred 0
pred 1
frame :  335
pred 0
pred 1
frame :  336
pred 0
pred 1
frame :  337
pred 0
pred 1
frame :  338
pred 0
pred 1
frame :  339
pred 0
pred 1
frame :  340
pred 0
pred 1
frame :  341
pred 0
pred 1
frame :  342
pred 0
pred 1
frame :  343
pred 0
pred 1
frame :  344
pred 0
pred 1
frame :  345
pred 0
pred 1
frame :  346
pred 0
pred 1
frame :  347
pred 0
pred 1
frame :  348
pred 1
pred 0
frame :  349
pred 1
pred 0
frame :  350
pred 1
pred 0
frame :  351
pred 1
pred 0
f

pred 1
pred 0
frame :  630
pred 1
pred 0
frame :  631
pred 1
pred 0
frame :  632
pred 1
pred 0
frame :  633
pred 1
pred 0
frame :  634
pred 1
pred 0
frame :  635
pred 1
pred 0
frame :  636
pred 1
pred 0
frame :  637
pred 1
pred 0
frame :  638
pred 1
pred 0
frame :  639
pred 1
pred 0
frame :  640
pred 1
pred 0
frame :  641
pred 1
pred 0
frame :  642
pred 1
pred 0
frame :  643
pred 1
pred 0
frame :  644
pred 1
pred 0
frame :  645
pred 1
pred 0
frame :  646
pred 1
pred 0
frame :  647
pred 1
pred 0
frame :  648
pred 1
pred 0
frame :  649
pred 1
pred 0
frame :  650
pred 1
pred 0
frame :  651
pred 1
pred 0
frame :  652
pred 1
pred 0
frame :  653
pred 1
pred 0
frame :  654
pred 1
pred 0
frame :  655
pred 1
pred 1
frame :  656
pred 1
pred 0
frame :  657
pred 1
pred 0
frame :  658
pred 1
pred 0
frame :  659
pred 1
pred 0
frame :  660
pred 1
pred 0
frame :  661
pred 1
pred 0
frame :  662
pred 1
pred 0
frame :  663
pred 1
pred 0
frame :  664
pred 1
pred 0
frame :  665
pred 1
pred 0
frame :  666
p

pred 1
pred 0
frame :  944
pred 0
pred 0
frame :  945
pred 0
frame :  946
pred 0
frame :  947
pred 0
frame :  948
pred 1
pred 0
frame :  949
pred 0
frame :  950
pred 1
frame :  951
pred 1
pred 0
frame :  952
pred 1
pred 0
frame :  953
pred 1
pred 0
frame :  954
pred 1
pred 0
frame :  955
pred 1
pred 0
frame :  956
pred 1
pred 0
frame :  957
pred 1
pred 0
frame :  958
pred 1
pred 0
frame :  959
pred 1
pred 0
frame :  960
pred 0
pred 1
frame :  961
pred 0
pred 1
frame :  962
pred 0
pred 1
frame :  963
pred 0
pred 1
frame :  964
pred 0
pred 1
frame :  965
pred 0
pred 1
frame :  966
pred 1
pred 0
frame :  967
pred 1
pred 0
frame :  968
pred 1
pred 0
frame :  969
pred 1
pred 0
frame :  970
pred 1
pred 0
frame :  971
pred 1
pred 0
frame :  972
pred 1
pred 0
frame :  973
pred 1
pred 0
frame :  974
pred 1
pred 0
frame :  975
pred 1
pred 0
frame :  976
pred 1
pred 0
frame :  977
pred 1
pred 0
frame :  978
pred 0
pred 1
frame :  979
pred 0
pred 1
frame :  980
pred 1
pred 0
frame :  981
pred 1
pr

...

pred 0
pred 1
frame :  2151
pred 0
pred 1
frame :  2152
pred 0
pred 1
frame :  2153
pred 1
pred 0
frame :  2154
pred 1
pred 0
frame :  2155
pred 0
pred 1
frame :  2156
pred 0
pred 1
frame :  2157
pred 0
pred 1
frame :  2158
pred 0
pred 1
frame :  2159
pred 0
pred 1
frame :  2160
pred 0
pred 1
frame :  2161
pred 0
pred 1
frame :  2162
pred 0
pred 1
frame :  2163
pred 0
pred 1
frame :  2164
pred 0
pred 1
frame :  2165
pred 0
pred 1
frame :  2166
pred 0
pred 1
frame :  2167
pred 0
pred 1
frame :  2168
pred 0
pred 1
frame :  2169
pred 0
pred 1
frame :  2170
pred 0
pred 1
frame :  2171
pred 0
pred 1
frame :  2172
pred 0
pred 1
frame :  2173
pred 0
pred 1
frame :  2174
pred 0
pred 1
frame :  2175
pred 0
pred 1
frame :  2176
pred 0
pred 1
frame :  2177
pred 0
pred 1
frame :  2178
pred 1
pred 0
frame :  2179
pred 0
pred 1
frame :  2180
pred 1
pred 0
frame :  2181
pred 1
pred 0
frame :  2182
pred 0
pred 1
frame :  2183
pred 0
pred 1
frame :  2184
pred 0
pred 1
frame :  2185
pred 0
pred 1
frame 

pred 2
pred 0
frame :  2452
pred 0
pred 0
frame :  2453
pred 1
pred 0
frame :  2454
pred 0
pred 0
frame :  2455
pred 1
pred 0
frame :  2456
pred 1
pred 0
frame :  2457
pred 1
pred 0
frame :  2458
pred 1
pred 0
frame :  2459
pred 1
pred 0
frame :  2460
pred 1
pred 0
frame :  2461
pred 1
pred 0
frame :  2462
pred 1
pred 0
frame :  2463
pred 1
pred 0
frame :  2464
pred 0
pred 0
frame :  2465
pred 0
pred 0
frame :  2466
pred 0
pred 1
frame :  2467
pred 0
pred 1
frame :  2468
pred 0
pred 1
frame :  2469
pred 0
pred 1
frame :  2470
pred 0
pred 1
frame :  2471
pred 0
pred 1
frame :  2472
pred 0
pred 1
frame :  2473
pred 0
pred 1
frame :  2474
pred 0
pred 1
frame :  2475
pred 0
pred 1
frame :  2476
pred 0
pred 1
frame :  2477
pred 0
pred 0
frame :  2478
pred 0
pred 0
frame :  2479
pred 0
pred 0
frame :  2480
pred 0
pred 0
frame :  2481
pred 0
pred 0
frame :  2482
pred 0
pred 0
frame :  2483
pred 0
pred 1
frame :  2484
pred 0
pred 0
frame :  2485
pred 0
pred 0
frame :  2486
pred 0
pred 1
frame 

pred 0
pred 1
frame :  2751
pred 0
pred 1
frame :  2752
pred 0
pred 1
frame :  2753
pred 0
pred 1
frame :  2754
pred 0
pred 1
frame :  2755
pred 0
pred 0
frame :  2756
pred 0
pred 1
frame :  2757
pred 0
pred 1
frame :  2758
pred 0
pred 1
frame :  2759
pred 1
pred 0
frame :  2760
pred 0
pred 1
frame :  2761
pred 0
pred 1
frame :  2762
pred 0
pred 1
frame :  2763
pred 0
pred 1
frame :  2764
pred 0
pred 1
frame :  2765
pred 0
pred 1
frame :  2766
pred 0
pred 1
frame :  2767
pred 0
pred 1
frame :  2768
pred 0
pred 1
frame :  2769
pred 0
pred 1
frame :  2770
pred 0
pred 1
frame :  2771
pred 0
pred 1
frame :  2772
pred 0
pred 1
frame :  2773
pred 0
pred 1
frame :  2774
pred 0
pred 1
frame :  2775
pred 0
pred 1
frame :  2776
pred 1
pred 0
frame :  2777
pred 1
pred 0
frame :  2778
pred 1
pred 0
frame :  2779
pred 1
pred 0
frame :  2780
pred 1
pred 0
frame :  2781
pred 1
pred 0
frame :  2782
pred 1
pred 0
frame :  2783
pred 1
pred 0
frame :  2784
pred 1
pred 0
frame :  2785
pred 1
pred 0
frame 

pred 1
pred 0
frame :  3052
pred 1
pred 0
frame :  3053
pred 1
pred 0
frame :  3054
pred 1
pred 0
frame :  3055
pred 1
pred 0
frame :  3056
pred 1
pred 0
frame :  3057
pred 1
pred 0
frame :  3058
pred 0
pred 0
frame :  3059
pred 0
pred 0
frame :  3060
pred 1
pred 0
frame :  3061
pred 1
pred 0
frame :  3062
pred 1
pred 0
frame :  3063
pred 1
pred 0
frame :  3064
pred 1
pred 0
frame :  3065
pred 1
pred 0
frame :  3066
pred 1
pred 0
frame :  3067
pred 1
pred 0
frame :  3068
pred 1
pred 0
frame :  3069
pred 1
pred 0
frame :  3070
pred 1
pred 0
frame :  3071
pred 1
pred 0
frame :  3072
pred 0
pred 0
frame :  3073
pred 0
pred 0
frame :  3074
pred 0
pred 0
frame :  3075
pred 1
pred 0
frame :  3076
pred 1
pred 0
frame :  3077
pred 1
pred 0
frame :  3078
pred 0
pred 0
frame :  3079
pred 1
pred 0
frame :  3080
pred 1
pred 0
frame :  3081
pred 1
pred 0
frame :  3082
pred 1
pred 0
frame :  3083
pred 1
pred 0
frame :  3084
pred 1
pred 0
frame :  3085
pred 1
pred 0
frame :  3086
pred 0
pred 1
frame 

pred 1
pred 0
frame :  3367
pred 1
pred 1
frame :  3368
pred 1
pred 1
frame :  3369
pred 1
pred 1
frame :  3370
pred 1
pred 1
frame :  3371
pred 1
pred 1
frame :  3372
pred 1
pred 1
frame :  3373
pred 1
pred 0
frame :  3374
pred 1
pred 0
frame :  3375
pred 1
pred 0
frame :  3376
pred 1
pred 1
frame :  3377
pred 1
pred 1
frame :  3378
pred 1
pred 1
frame :  3379
pred 1
pred 1
frame :  3380
pred 1
pred 1
frame :  3381
pred 1
pred 1
frame :  3382
pred 1
pred 1
frame :  3383
pred 1
pred 1
frame :  3384
pred 1
frame :  3385
pred 1
frame :  3386
pred 1
frame :  3387
pred 1
frame :  3388
pred 1
frame :  3389
pred 1
frame :  3390
pred 1
frame :  3391
pred 1
frame :  3392
pred 1
frame :  3393
pred 1
frame :  3394
pred 1
frame :  3395
pred 1
frame :  3396
pred 1
frame :  3397
pred 1
frame :  3398
pred 1
frame :  3399
pred 1
frame :  3400
pred 1
frame :  3401
pred 1
frame :  3402
pred 1
frame :  3403
pred 1
frame :  3404
pred 1
frame :  3405
pred 2
frame :  3406
pred 2
frame :  3407
pred 2
frame 